In [119]:
from socceraction.data.wyscout import PublicWyscoutLoader
from socceraction.spadl.wyscout import convert_to_actions
from socceraction.data.opta import OptaLoader
from socceraction.data.statsbomb import StatsBombLoader
from socceraction.spadl.config import actiontypes, bodyparts
import socceraction.spadl as spadl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from xgboost import XGBClassifier, XGBRegressor
import math

In [120]:
api_wyscout = PublicWyscoutLoader(root="data/wyscout")
api_opta = OptaLoader(root="data/opta")
# api_statsbomb = StatsBombLoader(root="data/statsbomb")

In [121]:
for idx, action_name in enumerate(actiontypes):
    print(f'action_id : {idx}   action_name : {action_name}')

action_id : 0   action_name : pass
action_id : 1   action_name : cross
action_id : 2   action_name : throw_in
action_id : 3   action_name : freekick_crossed
action_id : 4   action_name : freekick_short
action_id : 5   action_name : corner_crossed
action_id : 6   action_name : corner_short
action_id : 7   action_name : take_on
action_id : 8   action_name : foul
action_id : 9   action_name : tackle
action_id : 10   action_name : interception
action_id : 11   action_name : shot
action_id : 12   action_name : shot_penalty
action_id : 13   action_name : shot_freekick
action_id : 14   action_name : keeper_save
action_id : 15   action_name : keeper_claim
action_id : 16   action_name : keeper_punch
action_id : 17   action_name : keeper_pick_up
action_id : 18   action_name : clearance
action_id : 19   action_name : bad_touch
action_id : 20   action_name : non_action
action_id : 21   action_name : dribble
action_id : 22   action_name : goalkick


In [122]:
for idx, bodypart_name in enumerate(bodyparts):
    print(f'bodypart_id : {idx}   bodypart_name : {bodypart_name}')

bodypart_id : 0   bodypart_name : foot
bodypart_id : 1   bodypart_name : head
bodypart_id : 2   bodypart_name : other
bodypart_id : 3   bodypart_name : head/other
bodypart_id : 4   bodypart_name : foot_left
bodypart_id : 5   bodypart_name : foot_right


In [123]:
def convert_events_df_to_spadl(events_df, home_team_id):
    spadl_events_df = convert_to_actions(events_df, home_team_id)
    spadl_events_df['time_seconds'] = spadl_events_df['time_seconds'].astype('float64')
    spadl_events_df['timestamp'] = pd.to_datetime(spadl_events_df['time_seconds'], unit='s')
    spadl_events_df = spadl.play_left_to_right(spadl_events_df, home_team_id)
    return spadl_events_df

In [124]:
# FUNCTION TO ADD ADDITIONAL INFO IN RAW SPADL DATAFRAME
STANDARD_LENGTH_COURT = 105
STANDARD_WIDTH_COURT = 68
STANDARD_GOALLINE_WIDTH = 7.32

# Helper Functions
def calculate_distance_to_goal(length_court, width_court, coordinate_x, coordinate_y, is_home_team):
    if is_home_team:
        distance_to_goal = math.sqrt((abs(length_court - coordinate_x)) ** 2 + (abs((0.5 * width_court) - coordinate_y)) ** 2)
    else:
        distance_to_goal = math.sqrt((coordinate_x) ** 2 + (abs((0.5 * width_court) - coordinate_y)) ** 2)
    return distance_to_goal

def calculate_distance_to_goal_apply_df(row):
    return calculate_distance_to_goal(STANDARD_LENGTH_COURT, STANDARD_WIDTH_COURT, row['start_x'], row['start_y'], row['is_home_team'])

def calculate_angle_to_goal(goalline_width, length_court, width_court, coordinate_x, coordinate_y, is_home_team):
    if is_home_team:
        L = abs(length_court - coordinate_x)
    else:
        L = coordinate_x
    W = abs((0.5 * width_court) - coordinate_y)
    return math.atan((goalline_width * L) / (L ** 2 + W ** 2 - (goalline_width / 2) ** 2))

def calculate_angle_to_goal_apply_df(row):
    return calculate_angle_to_goal(STANDARD_GOALLINE_WIDTH, STANDARD_LENGTH_COURT, STANDARD_WIDTH_COURT, row['start_x'], row['start_y'], row['is_home_team'])

def filter_out_is_home_team_apply_df(row, home_team_id):
    return 1 if row['team_id'] == home_team_id else 0

# Add distance to goal column
def add_distance_to_goal_column_to_spadl_df(spadl_df):
    spadl_df['distance_to_goal'] = spadl_df.apply(calculate_distance_to_goal_apply_df, axis=1)
    return spadl_df

# Add angle to goal column 
def add_angle_to_goal_column_to_spadl_df(spadl_df):
    spadl_df['angle_to_goal'] = spadl_df.apply(calculate_angle_to_goal_apply_df, axis=1)
    return spadl_df

# Add is_home_team column (boolean 0/1)
def add_is_home_team_column_to_spadl_df(spadl_df, home_team_id):
    spadl_df['is_home_team'] = spadl_df.apply(lambda x : filter_out_is_home_team_apply_df(x, home_team_id), axis=1)
    return spadl_df

In [125]:
# Collect all dataset action specific type, export them to csv files
# Shot (action_id = 11), shot_penalty (action_id = 12), shot_freekick (action_id = 13)
SHOT_ACTION_ID = [11, 12, 13] 

def collect_raw_goal_spadl_df(source="Wyscout", period=1):
    api = api_wyscout
    list_competitions_ids = []
    list_game_ids = []

    competitions_df = api.competitions()
    for _, row in competitions_df.iterrows():
        list_competitions_ids.append((row['competition_id'], row['season_id']))
        
    for competition_id, season_id in list_competitions_ids:
        games_df = api.games(competition_id, season_id)
        for _, row in games_df.iterrows():
            list_game_ids.append((row['game_id'], row['home_team_id'], row['away_team_id']))
            
    for game_id, home_team_id, away_team_id in list_game_ids:
        this_game_events_df = api.events(game_id)
        this_game_events_spadl_df = convert_events_df_to_spadl(this_game_events_df, home_team_id)
        this_game_events_spadl_df = this_game_events_spadl_df[this_game_events_spadl_df['type_id'].isin(SHOT_ACTION_ID)]
        if (period != None):
            this_game_events_spadl_df = this_game_events_spadl_df[this_game_events_spadl_df['period_id'] == period]
        else:
            this_game_events_spadl_df = this_game_events_spadl_df[this_game_events_spadl_df['period_id'] == 1]
        # Add additional computed column to support xG model
        this_game_events_spadl_df = add_is_home_team_column_to_spadl_df(this_game_events_spadl_df, home_team_id)
        this_game_events_spadl_df = add_distance_to_goal_column_to_spadl_df(this_game_events_spadl_df)
        this_game_events_spadl_df = add_angle_to_goal_column_to_spadl_df(this_game_events_spadl_df)

        # Export to external csv iteratively
        this_game_events_spadl_df.to_csv(f'data/training_data_xgoal_wyscout/{game_id}_{home_team_id}_{away_team_id}_xgoal_data.csv')            

In [126]:
# MAIN DRIVER (comment it if csv files already loaded)
# collect_raw_goal_spadl_df()

In [127]:
# Load csv datas already retrieved then concat them into one big dataframe
import os

DIRECTORY_XGOAL_CSV_DATAS = "data/training_data_xgoal_wyscout"

def load_and_concat_xgoal_df_from_csv():
    list_pass_event_df = []
    for filename in os.listdir(DIRECTORY_XGOAL_CSV_DATAS):
        f = os.path.join(DIRECTORY_XGOAL_CSV_DATAS, filename)
        if os.path.isfile(f):
            pass_event_df = pd.read_csv(f)
            list_pass_event_df.append(pass_event_df)
    return pd.concat(list_pass_event_df)

In [128]:
big_dataframe_xgoal_model = load_and_concat_xgoal_df_from_csv()

In [129]:
# SELECT ONLY FEATURED COLUMN FROM BIG DATASETS
features_column_included = ["distance_to_goal", "angle_to_goal", "is_home_team", "bodypart_id", "result_id"]
big_dataframe_xpass_model = big_dataframe_xgoal_model[[c for c in big_dataframe_xgoal_model.columns if c in features_column_included]]
big_dataframe_xpass_model.head()

bodypart_id  result_id  is_home_team  distance_to_goal  angle_to_goal
0            0          0             1         17.123694       0.242346
1            0          0             0         75.905342       0.094651
2            0          0             0        100.912326       0.072426
3            3          0             0        101.077151       0.072191
4            0          0             1         28.682679       0.233111

In [130]:
# FEATURE PREPROCESSING BIG DATASETS AND CREATE XGBOOST MODEL
# 1. Change start_x, start_y, end_x, end_y with StandardScaler
scaler = preprocessing.StandardScaler()
columns = ["distance_to_goal", "angle_to_goal"]
big_dataframe_xgoal_model[columns] = scaler.fit_transform(big_dataframe_xgoal_model[columns])

# 2. Check if data is unbalanced. If it is unbalanced, then do method to oversize the sample
print(big_dataframe_xgoal_model['result_id'].value_counts())

# 3. Change result_id label into float64 type
big_dataframe_xgoal_model['result_id'] = big_dataframe_xgoal_model['result_id'].astype('float64')

# 4. Remove dataframe instead of having result_id (0,1) --> (fail, success)
big_dataframe_xgoal_model = big_dataframe_xgoal_model[big_dataframe_xgoal_model['result_id'].isin([0,1])]
print(big_dataframe_xgoal_model['result_id'].value_counts())

# 5. Split train data and test data from Big Datasets
all_feature_columns = ["distance_to_goal", "angle_to_goal", "is_home_team", "bodypart_id"]
X_train = big_dataframe_xgoal_model[all_feature_columns]
Y_train = big_dataframe_xgoal_model["result_id"]
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, Y_train, test_size=0.4)

# 6. Train XGBoost Model 
modelXGB = XGBRegressor()
modelXGB.fit(X_train_split, y_train_split)

# 7. Predict Testing Data
y_predict = modelXGB.predict(X_test_split)

# 8. Display report for prediction
print("Mean squared error : ", mean_squared_error(y_test_split, y_predict))

0    18981
1     2185
Name: result_id, dtype: int64
0.0    18981
1.0     2185
Name: result_id, dtype: int64
Mean squared error :  0.08655013477989461
